In [2]:
import pandas as pd

# Load the uploaded dataset to inspect the columns and data types
file_path = "C:/Users/saidh/Downloads/rotten_tomatoes_critic_reviews/rotten_tomatoes_critic_reviews.csv"
data = pd.read_csv(file_path)

# Display the first few rows and summary information
data.head(), data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130017 entries, 0 to 1130016
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   rotten_tomatoes_link  1130017 non-null  object
 1   critic_name           1111488 non-null  object
 2   top_critic            1130017 non-null  bool  
 3   publisher_name        1130017 non-null  object
 4   review_type           1130017 non-null  object
 5   review_score          824081 non-null   object
 6   review_date           1130017 non-null  object
 7   review_content        1064211 non-null  object
dtypes: bool(1), object(7)
memory usage: 61.4+ MB


(  rotten_tomatoes_link      critic_name  top_critic           publisher_name  \
 0            m/0814255  Andrew L. Urban       False           Urban Cinefile   
 1            m/0814255    Louise Keller       False           Urban Cinefile   
 2            m/0814255              NaN       False      FILMINK (Australia)   
 3            m/0814255     Ben McEachen       False  Sunday Mail (Australia)   
 4            m/0814255      Ethan Alter        True       Hollywood Reporter   
 
   review_type review_score review_date  \
 0       Fresh          NaN  2010-02-06   
 1       Fresh          NaN  2010-02-06   
 2       Fresh          NaN  2010-02-09   
 3       Fresh        3.5/5  2010-02-09   
 4      Rotten          NaN  2010-02-10   
 
                                       review_content  
 0  A fantasy adventure that fuses Greek mythology...  
 1  Uma Thurman as Medusa, the gorgon with a coiff...  
 2  With a top-notch cast and dazzling special eff...  
 3  Whether audiences will g

In [3]:
# 1. Count of missing values in each column
na_counts = data.isna().sum()

# 2. Unique values for each column
unique_values = {col: data[col].unique() for col in data.columns}

# 3. Data type summary
data_info = data.dtypes

# Display results
print("Missing Values per Column:\n", na_counts)
print("\nUnique Values per Column:")
for col, values in unique_values.items():
    print(f"Column '{col}': {values[:10]}...")  # Display only the first 10 unique values for brevity

print("\nData Types of Columns:\n", data_info)

Missing Values per Column:
 rotten_tomatoes_link         0
critic_name              18529
top_critic                   0
publisher_name               0
review_type                  0
review_score            305936
review_date                  0
review_content           65806
dtype: int64

Unique Values per Column:
Column 'rotten_tomatoes_link': ['m/0814255' 'm/0878835' 'm/10' 'm/1000013-12_angry_men'
 'm/1000079-20000_leagues_under_the_sea' 'm/10000_bc' 'm/1000121-39_steps'
 'm/1000123-310_to_yuma' 'm/10002008-charly' 'm/1000204-abraham_lincoln']...
Column 'critic_name': ['Andrew L. Urban' 'Louise Keller' nan 'Ben McEachen' 'Ethan Alter'
 'David Germain' 'Nick Schager' 'Bill Goodykoontz' 'Jordan Hoffman'
 'Jim Schembri']...
Column 'top_critic': [False  True]...
Column 'publisher_name': ['Urban Cinefile' 'FILMINK (Australia)' 'Sunday Mail (Australia)'
 'Hollywood Reporter' 'Associated Press' 'Slant Magazine'
 'Arizona Republic' 'UGO' 'The Age (Australia)' 'Daily Mirror (UK)']...
Column 

In [4]:
import re

# Step 1: Drop rows with missing `review_content` (essential for sentiment analysis)
data_cleaned = data.dropna(subset=['review_content'])

# Step 2: Handle missing values in `review_score`
# Attempt to standardize `review_score` to numeric values where possible
def extract_score(score):
    if pd.isna(score):
        return None
    # Extract numeric part, e.g., "3.5/5" -> 3.5, "70%" -> 70, or return None for non-standard formats
    match = re.match(r'(\d+(\.\d+)?)', str(score))
    return float(match.group(1)) if match else None

data_cleaned['review_score'] = data_cleaned['review_score'].apply(extract_score)

# Step 3: Convert `review_date` to datetime format
data_cleaned['review_date'] = pd.to_datetime(data_cleaned['review_date'], errors='coerce')

# Step 4: Drop rows with missing values in columns still necessary for analysis
data_cleaned = data_cleaned.dropna(subset=['critic_name', 'review_type', 'review_date'])

# Step 5: Select relevant columns for analysis
columns_to_keep = [
    'rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name', 
    'review_type', 'review_score', 'review_date', 'review_content'
]
data_final = data_cleaned[columns_to_keep]

C:\Users\saidh\AppData\Local\Temp\ipykernel_3928\2785744938.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['review_score'] = data_cleaned['review_score'].apply(extract_score)
C:\Users\saidh\AppData\Local\Temp\ipykernel_3928\2785744938.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['review_date'] = pd.to_datetime(data_cleaned['review_date'], errors='coerce')


In [5]:
# Display the data types of each column
print("Data Types of Columns in data_final:\n", data_final.dtypes)

# Check for any rows with mixed types in each column by verifying consistent data types
# Summarize data type checks
for column in data_final.columns:
    unique_types = data_final[column].apply(type).nunique()
    if unique_types > 1:
        print(f"Column '{column}' contains mixed data types.")
    else:
        print(f"Column '{column}' has consistent data type.")

# Display the first few rows of the dataframe for a quick inspection
print("\nSample Data:\n", data_final.head())


Data Types of Columns in data_final:
 rotten_tomatoes_link            object
critic_name                     object
top_critic                        bool
publisher_name                  object
review_type                     object
review_score                   float64
review_date             datetime64[ns]
review_content                  object
dtype: object
Column 'rotten_tomatoes_link' has consistent data type.
Column 'critic_name' has consistent data type.
Column 'top_critic' has consistent data type.
Column 'publisher_name' has consistent data type.
Column 'review_type' has consistent data type.
Column 'review_score' has consistent data type.
Column 'review_date' has consistent data type.
Column 'review_content' has consistent data type.

Sample Data:
   rotten_tomatoes_link      critic_name  top_critic           publisher_name  \
0            m/0814255  Andrew L. Urban       False           Urban Cinefile   
1            m/0814255    Louise Keller       False           Urban Ci

In [16]:
# Save the cleaned data
output_path = 'C:/Users/saidh/Downloads/rotten_tomatoes_critic_reviews/rotten_tomatoes_critic_reviews_cleaned.csv'
data_final.to_csv(output_path, index=False)

In [6]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1054198 entries, 0 to 1130016
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   rotten_tomatoes_link  1054198 non-null  object        
 1   critic_name           1054198 non-null  object        
 2   top_critic            1054198 non-null  bool          
 3   publisher_name        1054198 non-null  object        
 4   review_type           1054198 non-null  object        
 5   review_score          630853 non-null   float64       
 6   review_date           1054198 non-null  datetime64[ns]
 7   review_content        1054198 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(1), object(5)
memory usage: 65.3+ MB


In [1]:
!pip install google-cloud-language pandas


   ---------------------------------------- 0.0/153.8 kB ? eta -:--:--
   --------------- ------------------------ 61.4/153.8 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 153.8/153.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/156.5 kB ? eta -:--:--
   ---------------------------------------- 156.5/156.5 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/209.5 kB ? eta -:--:--
   --------------------------------------- 209.5/209.5 kB 12.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.1 kB ? eta -:--:--
   ---------------------------------------- 50.1/50.1 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/431.5 kB ? eta -:--:--
   ---------------------------------------  430.1/431.5 kB 8.9 MB/s eta 0:00:01
   ---------------------------------------- 431.5/431.5 kB 9.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/220.9 kB ? eta -:--:--
   ---------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.12.0 requires protobuf<=3.20.1,>=3.12.2, but you have protobuf 5.28.3 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.28.3 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.28.3 which is incompatible.


In [ ]:
import os

# Set the environment variable for authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/saidh/Downloads/fa24-i535-ksdhanus-moviereview-6f36c0289e17.json"

In [13]:
import pandas as pd
from google.cloud import language_v1
from tqdm import tqdm
import os

# Set up the Google Language API client
client = language_v1.LanguageServiceClient()

# Load the reviews dataset
data = pd.read_csv('C:/Users/saidh/Downloads/rotten_tomatoes_critic_reviews/rotten_tomatoes_critic_reviews_cleaned.csv')

# Function to analyze sentiment of text
def analyze_sentiment(text):
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT, language="en")
    sentiment = client.analyze_sentiment(request={"document": document}).document_sentiment
    return sentiment.score, sentiment.magnitude

# Process data in batches and add a progress bar
batch_size = 500  # Set batch size, e.g., 500 reviews per batch
results = []

print("Starting sentiment analysis on review content...")

for start in tqdm(range(0, len(data), batch_size)):
    batch = data.iloc[start:start+batch_size]
    batch_results = batch['review_content'].apply(lambda x: analyze_sentiment(str(x)) if pd.notnull(x) else (None, None))
    results.extend(batch_results)

# Split results into two columns and assign to DataFrame
sentiment_scores, sentiment_magnitudes = zip(*results)
data['sentiment_score'] = sentiment_scores
data['sentiment_magnitude'] = sentiment_magnitudes

# Display a sample of the DataFrame to verify results
data.head()

Starting sentiment analysis on review content...


  0%|▎                                                                             | 9/2109 [08:39<33:40:59, 57.74s/it]


ResourceExhausted: 429 Quota exceeded for quota metric 'Requests' and limit 'Requests per minute' of service 'language.googleapis.com' for consumer 'project_number:612483773688'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "language.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "language.googleapis.com/default_requests"
}
metadata {
  key: "quota_location"
  value: "global"
}
metadata {
  key: "quota_limit"
  value: "DefaultRequestsPerMinutePerProject"
}
metadata {
  key: "quota_limit_value"
  value: "600"
}
metadata {
  key: "consumer"
  value: "projects/612483773688"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quotas/help/request_increase"
}
]